In [ ]:
pip install openpyxl


Looking in indexes: http://mirrors.aliyun.com/pypi/simple/
  Using cached http://mirrors.aliyun.com/pypi/packages/c0/da/977ded879c29cbd04de313843e76868e6e13408a94ed6b987245dc7c8506/openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
  Using cached http://mirrors.aliyun.com/pypi/packages/c1/8b/5fe2cc11fee489817272089c4203e679c63b570a5aaeb18d852ae3cbba6a/et_xmlfile-2.0.0-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install transformers

Looking in indexes: http://mirrors.aliyun.com/pypi/simple/
  Using cached http://mirrors.aliyun.com/pypi/packages/96/88/beb33a79a382fcd2aed0be5222bdc47f41e4bfe7aaa90ae1374f1d8ea2af/transformers-4.53.2-py3-none-any.whl (10.8 MB)
  Using cached http://mirrors.aliyun.com/pypi/packages/46/7b/98daa50a2db034cab6cd23a3de04fa2358cb691593d28e9130203eb7a805/huggingface_hub-0.33.4-py3-none-any.whl (515 kB)
  Using cached http://mirrors.aliyun.com/pypi/packages/13/c3/cc2755ee10be859c4338c962a35b9a663788c0c0b50c0bdd8078fb6870cf/tokenizers-0.21.2-cp39-abi3-win_amd64.whl (2.5 MB)
  Using cached http://mirrors.aliyun.com/pypi/packages/69/e2/b011c38e5394c4c18fb5500778a55ec43ad6106126e74723ffaee246f56e/safetensors-0.5.3-cp38-abi3-win_amd64.whl (308 kB)
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
import os

# 第一步导入数据
# 检查文件是否存在
file_path = 'week10/jd_comment_data.xlsx'
if os.path.exists(file_path):
    df = pd.read_excel(file_path)
    #print(df.head())
else:
    print(f"文件 {file_path} 不存在！")



文件 week10/jd_comment_data.xlsx 不存在！


In [7]:

#第二步 过滤数据只取出需要的数据
filterd = df[(df['评价内容(content)'].str.len() > 5) & (df['评价内容(content)'] != "此用户未填写评价内容")]
data_df = filterd[['评价内容(content)','评分（总分5分）(score)']]

In [12]:
data=data_df.values

In [13]:
#第三步拆分训练集与测试集
from sklearn.model_selection import train_test_split
train ,test = train_test_split(data)

In [17]:
#第四步，加分词器，并将语料处理为Datalord
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenzier = AutoTokenizer.from_pretrained('hfl/chinese-bert-wwm')

d:\software\python\miniconda\envs\py312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\software\python\miniconda\envs\py312\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\wzy\.cache\huggingface\hub\models--hfl--chinese-bert-wwm. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate

In [ ]:
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
def warp_data(batch_data):
    comments, lables = [],[]
    for bdate in batch_data:
        comments.append(bdate[0])
        lables.append(int(bdate[1])-1)  # 标签取值[0-4]
    
    # 转换模型输入数据,pt表示的就是PyTorch，truncation表示如果超出了最大限制就截取，为false超出就报错
    input_data = tokenzier(comments, return_tensors='pt', padding=True, truncation=True, max_length=512)
    labels_data = torch.tensor(lables)
    
    return input_data, labels_data

train_dl = DataLoader(train, batch_size=20, shuffle=True, collate_fn = warp_data)
test_dl = DataLoader(test, batch_size=20, shuffle=False, collate_fn = warp_data)

In [26]:
#第五步，新建模型、损失函数与学习率
model_1 = AutoModelForSequenceClassification.from_pretrained('hfl/chinese-bert-wwm', num_labels=5)
model_2 = AutoModelForSequenceClassification.from_pretrained('hfl/chinese-bert-wwm', num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/chinese-bert-wwm and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/chinese-bert-wwm and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
from torch.optim import Adam
# loss、optim
loss_fn1 = nn.CrossEntropyLoss()
optim1 = Adam(model_1.parameters(), lr=1e-3)

loss_fn2 = nn.CrossEntropyLoss()
optim2 = Adam(model_2.parameters(), lr=1e-3)

In [28]:
from tqdm import tqdm #进度条函数
from torch.utils.tensorboard import SummaryWriter  #TensorBoard可视化配置

writer = SummaryWriter()
model1_train_loss_cnt = 0

for epoch in range(3):
    pbar = tqdm(train_dl)
    for input_data, labels_data in pbar:
        datas = { k:v for k,v in input_data.items() }
        labels = labels_data
        
        result = model_1(**datas)
        loss = loss_fn1(result.logits, labels)
        
        pbar.set_description(f'epoch:{epoch} train_loss:{loss.item():.4f}')

        writer.add_scalar("Fine Tuning Train Loss", loss, model1_train_loss_cnt)
        model1_train_loss_cnt += 1
        
        loss.backward()
        optim1.step()
        
        model_1.zero_grad()


torch.save(model_1.state_dict(),'model_1.pt')

epoch:0 train_loss:0.6201:   1%|▏         | 22/1506 [03:57<4:27:18, 10.81s/it]


KeyboardInterrupt: 

In [ ]:
for param in model_2.parameters():
    param.requires_grad = False


model2_train_loss_cnt = 0

for epoch in range(3):
    pbar = tqdm(train_dl)
    for input_data, labels_data in pbar:
        datas = { k:v for k,v in input_data.items() }
        labels = labels_data
        
        result = model_2(**datas)
        loss = loss_fn2(result.logits, labels)
        
        pbar.set_description(f'epoch:{epoch} train_loss:{loss.item():.4f}')

        writer.add_scalar("Transfer Learning Train Loss", loss, model2_train_loss_cnt)
        model2_train_loss_cnt += 1
        
        loss.backward()
        optim2.step()
        
        model_2.zero_grad()


torch.save(model_2.state_dict(),'model_2.pt')

In [ ]:
model_1.eval()
model_2.eval()
pbar = tqdm(test_dl)


correct1, correct2 = 0,0

for input_data, labels_data in pbar:
    datas = { k:v for k,v in input_data.items() }
    labels = labels_data

    with torch.no_grad():
        result1 = model_1(**datas)
        result2 = model_2(**datas)

    predict1 = torch.argmax(result1.logits, dim=-1)
    predict2 = torch.argmax(result2.logits, dim=-1)

    correct1 += (predict1 == labels).sum()
    correct2 += (predict1 == labels).sum()